# **Main script**

## Step 1. Import libraries

In [ ]:
!conda install pandas --yes

In [ ]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests 
#from bs4 import BeautifulSoup
#import html5lib

In [ ]:
import itertools

import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter

import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline
# notice: installing seaborn might takes a few minutes
!conda install -c anaconda seaborn -y
import seaborn as sns

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.11.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

In [ ]:
!pip install geocoder #get latitude and longtitude if know address/postalcode

In [ ]:
!conda install -c districtdatalabs yellowbrick --yes

In [ ]:
!pip install lxml

In [ ]:
!pip install BeautifulSoup4
from bs4 import BeautifulSoup

## Step 2. Import datas

In [ ]:
import html5lib

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_T'

dfs = pd.read_html(url)
df = dfs[1] #import the nth table 
df.reset_index(drop=True, inplace=True)
df

In [ ]:
df.dtypes

In [ ]:
type(df.index)

In [ ]:
df.shape

Only consider Edmonton related data

In [ ]:
df_EDMONTON = df[df['Borough'] == 'Edmonton'].reset_index(drop=True)
df_EDMONTON.tail()

## Step 3. Clean data

3.1 Let dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [ ]:
df=df.rename(columns={"Postal Code": "PostalCode"})
df.head()

3.2 Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned

In [ ]:
df=df[df['Borough']!='Not assigned']
df.index = range(len(df))
df.tail()

3.3 Only process the cells that have an assigned LATITUDE or LONGITUDE. Ignore cells with a borough that is Not assigned.

In [ ]:
df=df[df['Latitude']!='Not assigned']
df.index = range(len(df))
df.tail()

3.4 If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [ ]:
#check if there any row contains "Not assigned" Neighborhood
df1 = df[df['Neighborhood'].str.contains("Not assigned")]
#check numbers of rows in the df1 dataframe contains "Not assigned" Neighborhood
a = len(df1.index) 

if a > 0:
    df.loc[df['Neighborhood'] == 'Not assigned','Neighborhood'] = df['Borough']
    df.head()
else:
    print("At current step, no row in the dataframe contains Not assigned neighborhood.")
    print(df.tail())

3.5 More than one neighborhood can exist in one postal code area. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table

In [ ]:
#check whether there is repeated postalcode exist in the current dataframe
df2 = df.copy()
df3 = df2.drop_duplicates('PostalCode')
a2 = len(df2.index) 
a3 = len(df3.index) 
print('The original dataframe contains '+str(a2)+' rows and the simplyfied dataframe contains '+str(a3)+' rows')

if a3 == a2:
    print("The current dataframe is good to go. We may estimate the number of rows in the dataframe.")
    df_new=df
else:
    print("We need to further clean the data.")
    df_clean = df.groupby(['Postalcode','Borough'], sort=False).agg( ', '.join)
    print(df_clean.head())
    df_new=df_clean.reset_index() #add back the index which is 0,1,2,3,4,5,.... to the first column

In [ ]:
df_new

## Step 4. Visualization the neighborhoods in Edmonton    

4.1 Obtain only Edmonton related data

In [ ]:
df_EDMONTON = df[df['Borough'] == 'Edmonton'].reset_index(drop=True)
df_EDMONTON

In [ ]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_EDMONTON['Borough'].unique()),
        df_EDMONTON.shape[0]
    ))

In [ ]:
df_EDMONTON.dtypes

In [ ]:
df_EDMONTON['Latitude'] = df_EDMONTON['Latitude'].astype(float)
df_EDMONTON['Longitude'] = df_EDMONTON['Longitude'].astype(float)

4.2 Get the geographical coordinates for each borough in Edmonton

In [ ]:
address = 'Edmonton, AB'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Edmonton are {}, {}.'.format(latitude, longitude))

4.3 Visualize neighborhoods of Edmonton on map

In [ ]:
neighborhoods=df_EDMONTON.copy()
# create map of Edmonton using latitude and longitude values
map_edmonton = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=9,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_edmonton)  
    
map_edmonton

In [ ]:
!pip install geopandas

### 4.0 HEAT MAP - FSA

In [ ]:
import geopandas as gpd
response = requests.get(r"https://github.com/simon8907/FSA_Data/raw/master/Canadian_FSA.geojson")
data = response.json()

NeisA = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')
NeisA.tail()

In [ ]:
df_FSAedm=df_EDMONTON.copy()
df_FSAedm['CFSAUID_ID']=list(range(1,(len(df_FSAedm)+1)))
df_FSAedm.rename(columns={'PostalCode':'CFSAUID'}, inplace=True)
df_FSAedm.tail()

In [ ]:
#NeisA.rename(columns={'name':'Neighbourhood'}, inplace=True)
NeiAmerge = pd.merge(NeisA,
                df_FSAedm[['CFSAUID','CFSAUID_ID']],
                 on='CFSAUID')
NeiAmerge['centroid_long']=NeiAmerge.centroid.x
NeiAmerge['centroid_lag']=NeiAmerge.centroid.y
NeiAmerge.tail()

In [ ]:
import branca.colormap as cm
#colormap = cm.linear.YlGnBu_09
colormap = cm.linear.Paired_12.scale(NeiAmerge['CFSAUID_ID'].min(),NeiAmerge['CFSAUID_ID'].max()).to_step(NeiAmerge['CFSAUID_ID'].max()) 
colormap

In [ ]:
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup


# create a plain world map
edmA_map = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='CartoDB positron') #, tiles = 'Stamen Toner')


style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']['CFSAUID_ID']), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}


g = folium.GeoJson(
    NeiAmerge,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['CFSAUID','CFSAUID_ID'],
        aliases=['PostalCode','PostalCode_ID'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
        sticky=True
        )).add_to(edmA_map)

colormap.add_to(edmA_map)

edmA_map

In [ ]:
from branca.element import Template, MacroElement
class DivIcon(MacroElement):
    def __init__(self, html='', size=(1,1), anchor=(0,0), style=''):
        """TODO : docstring here"""
        super(DivIcon, self).__init__()
        self._name = 'DivIcon'
        self.size = size
        self.anchor = anchor
        self.html = html
        self.style = style

        self._template = Template(u"""
            {% macro header(this, kwargs) %}
              <style>
                .{{this.get_name()}} {
                    {{this.style}}
                    }
              </style>
            {% endmacro %}
            {% macro script(this, kwargs) %}
                var {{this.get_name()}} = L.divIcon({
                    className: '{{this.get_name()}}',
                    iconSize: [{{ this.size[0] }},{{ this.size[1] }}],
                    iconAnchor: [{{ this.anchor[0] }},{{ this.anchor[1] }}],
                    html : "{{this.html}}",
                    });
                {{this._parent.get_name()}}.setIcon({{this.get_name()}});
            {% endmacro %}
            """)
for i in range(len(NeiAmerge)):
    lag=NeiAmerge.iloc[i][6]
    log=NeiAmerge.iloc[i][5]
    text=NeiAmerge.iloc[i][1]
    folium.map.Marker(
        [lag, log],
        icon=DivIcon(
            size=(1,1),
            anchor=(0,0),
            html=text,
            style="""
            font-size:12px;
            background-color: transparent;
            border-color: transparent;
            text-align: center;
            """
        )
    ).add_to(edmA_map)
edmA_map

**---------------------------------------------------------------------------------------------------------------------------------------**

### **According to City of Edmonton, Edmonton is shaped using political Ward boundaries or Neibourhood boundaries**  (Not based on postal code)
references:
 https://data.edmonton.ca/browse?category=Geospatial+Boundaries&page=2 search page
 - https://data.edmonton.ca/Geospatial-Boundaries/City-of-Edmonton-Ward-Boundaries-effective-to-23-5/yhng-294h Ward related geo-information
 - https://data.edmonton.ca/Geospatial-Boundaries/City-of-Edmonton-Neighbourhood-Boundaries/jfvj-x253 Neighbourhood related geo-information
 - https://data.edmonton.ca/dataset/City-Of-Edmonton-Neighbourhood-Boundaries-Spatial-/tvcx-3vrx Neighbourhood related geo-information
 - https://dashboard.edmonton.ca/Geospatial-Boundaries/City-of-Edmonton-Neighbourhood-Boundaries/jfvj-x253 Neighbourhood related geo-information
 - https://dashboard.edmonton.ca/Geospatial-Boundaries/City-of-Edmonton-Neighbourhood-Boundaries/jfvj-x253/data Neighbourhood related geo-info
 - https://data.edmonton.ca/Administrative/zone-53-map/6v4j-wnrz Neighbourhood related geo-info
 - https://data.edmonton.ca/Geospatial-Boundaries/Mature-Neighbourhoods-Map/3jmw-i9z8 only present mature neighbourhood geo-info
 - https://www.edmonton.ca/city_government/municipal_elections/civic-election-maps.aspx Ward map in pdf only
 - https://www.edmonton.ca/city_government/documents/WBC_Final_Report_2020.pdf Ward information in pdf
 - https://data.edmonton.ca/Geospatial-Boundaries/Mosquito-Program-Boundaries/esxr-3x66 only present the city and its nearby cities

### **4.1 Visualization of Wards**

In [ ]:
import pandas as pd

dataward = {'Ward':  ['WARD 01', 'WARD 02','WARD 03', 'WARD 04','WARD 05', 'WARD 06','WARD 07', 'WARD 08','WARD 09', 'WARD 10','WARD 11', 'WARD 12'],
        'WardID': [1,2,3,4,5,6,7,8,9,10,11,12],
        }

df_ward = pd.DataFrame (dataward, columns = ['Ward','WardID'])
df_ward.head()

In [ ]:
df_ward.dtypes

In [ ]:
# download edmonton geojson file
#!wget --quiet https://data.edmonton.ca/api/geospatial/yhng-294h?method=export&format=GeoJSON -O edmonton.json
!pip install wget
import wget
wget.download('https://data.edmonton.ca/api/geospatial/yhng-294h?method=export&format=GeoJSON', 'edmonton.json')
    
print('GeoJSON file downloaded!')

In [ ]:
edmonton_geo = r'edmonton.json' ######## geojson file

**--------------------------------------------------------------------------------------------------------------------------------------**

**The following lines are just some examples to load GeoJson file**


!wget --quiet https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DV0101EN/labs/Data_Files/world_countries.json -O # download countries all over the world geojson fileworld_countries.json
    
print('GeoJSON file downloaded!')

world_geo = r'world_countries.json' ######## geojson file

!wget --quiet https://cocl.us/sanfran_geojson -O sanfran.json # download countries geojson file
    
print('GeoJSON file downloaded!')

sanfran_geo = r'sanfran.json' # geojson file

**--------------------------------------------------------------------------------------------------------------------------------------**

**Following code is code to open and read the GeoJson files**

import urllib.request
import urllib, json

with urllib.request.urlopen("https://data.edmonton.ca/api/geospatial/yhng-294h?method=export&format=GeoJSON") as url:
    edmonton_geo = url.read()
    ######### this would output the html source code
    #########print(edmonton_geo)

import urllib.request
import urllib, json

with urllib.request.urlopen("https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DV0101EN/labs/Data_Files/world_countries.json") as url:
    world_geo = url.read()
    ######## this would output the html source code
    ########print(world_geo)

**--------------------------------------------------------------------------------------------------------------------------------------**

In [ ]:
# create a plain world map
edm_map = folium.Map(location=[latitude, longitude], zoom_start=10) #, tiles = 'Stamen Toner')

In [ ]:
# display map
#edm_map   # un-# if wanna see map

In [ ]:
# create a numpy array of length 6 and has linear spacing from the minium total immigration to the maximum total immigration
#threshold_scale = np.linspace(df_ward['WardID'].min(),
#                              df_ward['WardID'].max(),
#                              6, dtype=int)
#threshold_scale = threshold_scale.tolist() # change the numpy array to a list
#threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration

# generate choropleth map 
edm_map.choropleth(
    geo_data=edmonton_geo,
    data=df_ward,
    columns=['Ward', 'WardID'],
    key_on='feature.properties.name',
    #threshold_scale=threshold_scale,
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Wards in Edmonton'
)

# display map
edm_map

In [ ]:

import geopandas as gpd
response = requests.get(r"https://data.edmonton.ca/api/geospatial/yhng-294h?method=export&format=GeoJSON")
data = response.json()
wards = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

wards.head()

In [ ]:
wards.rename(columns={'name':'Ward'}, inplace=True)
wardsmerge = pd.merge(wards,
                df_ward[['Ward','WardID']],
                 on='Ward')
wardsmerge['centroid_long']=wardsmerge.centroid.x
wardsmerge['centroid_lag']=wardsmerge.centroid.y
wardsmerge

In [ ]:
import branca.colormap as cm
#colormap = cm.linear.YlGnBu_09
colormap = cm.linear.Set1_09.scale(df_ward['WardID'].min(), df_ward['WardID'].max()).to_step(12) 
colormap

In [ ]:
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

# create a plain world map
edmward_map = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='CartoDB positron') #, tiles = 'Stamen Toner')


style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']['WardID']), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}



# tooltip=folium.features.GeoJsonTooltip(fields=['Ward','WardID'],
#            aliases=['Ward','Ward number'],
#            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
#            sticky=True
#        )


g = folium.GeoJson(
    wardsmerge,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Ward','WardID'],
        aliases=['Ward','Ward number'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
        sticky=True
        )).add_to(edmward_map)

colormap.add_to(edmward_map)

edmward_map

In [ ]:
class DivIcon(MacroElement):
    def __init__(self, html='', size=(5,5), anchor=(0,0), style=''):
        """TODO : docstring here"""
        super(DivIcon, self).__init__()
        self._name = 'DivIcon'
        self.size = size
        self.anchor = anchor
        self.html = html
        self.style = style

        self._template = Template(u"""
            {% macro header(this, kwargs) %}
              <style>
                .{{this.get_name()}} {
                    {{this.style}}
                    }
              </style>
            {% endmacro %}
            {% macro script(this, kwargs) %}
                var {{this.get_name()}} = L.divIcon({
                    className: '{{this.get_name()}}',
                    iconSize: [{{ this.size[0] }},{{ this.size[1] }}],
                    iconAnchor: [{{ this.anchor[0] }},{{ this.anchor[1] }}],
                    html : "{{this.html}}",
                    });
                {{this._parent.get_name()}}.setIcon({{this.get_name()}});
            {% endmacro %}
            """)
for i in range(len(wardsmerge)):
    lag=wardsmerge.iloc[i][5]
    log=wardsmerge.iloc[i][4]
    text=wardsmerge.iloc[i][2]
    folium.map.Marker(
        [lag, log],
        icon=DivIcon(
            size=(5,5),
            anchor=(0,0),
            html=text,
            style="""
            font-size:12px;
            background-color: transparent;
            border-color: transparent;
            text-align: center;
            """
        )
    ).add_to(edmward_map)
edmward_map

### **4.2 Visualization of Neighbourhoods**

Please refers to section 5.2

## Step 5. Data sorting and cleanning -- Wards and corresponding Neighbourhood

**------------------------------------------------------------------------------------------------------------------------------------**

### 5.0 Import data from any opensource    
(These sources cannot be used to correspond wards and neighbourhoods, since Ward boundaries update annually. Data in this section is in year 2012, 2014, 2016 and 2019.)

**Information for each ward in Edmonton**    
reference:  
 - https://www.epl.ca/blogs/post/edmonton-ward-profiles/ (2012 census- Dwelling type data)
 - https://data.edmonton.ca/Census/2014-Census-Population-By-Citizenship-Neighbourhoo/g2i5-7nqy (2014 census- Population)
 - https://data.edmonton.ca/Census/2012-Census-Dwelling-Unit-By-Structure-Type-Neighb/achm-af7d/data (2016 census- Dwelling type data)
 - https://data.edmonton.ca/Census/2016-Census-Population-by-Age-Range-Neighbourhood-/phd4-y42v (2016 census- Population by Age Range data)
 - https://data.edmonton.ca/Census/2016-Census-Population-by-Age-Range-Neighbourhood-/ham9-4niv (2016 census- Population by Age Range data)
 - https://data.edmonton.ca/Census/2019-Census-Population-by-Age-Range-Neighbourhood-/a6zx-dzqn (2019 census- Population by Age Range data)
 - https://data.edmonton.ca/Geospatial-Boundaries/Neighbourhoods-and-Wards/gihh-utrc exact Neighbourhood and Wards correspomding data

#df_N8BR = pd.read_csv("https://data.edmonton.ca/api/views/achm-af7d/rows.csv?accessType=DOWNLOAD") #2012 census data
#df_N8BR = pd.read_csv("https://data.edmonton.ca/api/views/phd4-y42v/rows.csv?accessType=DOWNLOAD")  #2016 census data
df_N8BR = pd.read_csv("https://data.edmonton.ca/api/views/a6zx-dzqn/rows.csv?accessType=DOWNLOAD") #2019 census data

df_N8BR.tail(3)

#df_N8BR.drop(['NEIGHBOURHOOD_NUMBER','SINGLE_DETACHED_HOME','DUPLEX/FOURPLEX','ROW_HOUSE','APARTMENT_(5+_STORIES)','APARTMENT_(1-4_STORIES)','MANUFACTURED/MOBILE_HOME','INSTITUTION/COLLECTIVE_RESIDENCE','HOTEL/MOTEL','RV/TENT/OTHER','NO_RESPONSE'],axis=1, inplace=True)
df_N8BR = df_N8BR[['Ward','Neighbourhood Name']]
df_N8BR.rename(columns={'Neighbourhood Name':'Neighbourhood'}, inplace=True)
df_N8BR.tail()

df_N8BR = df_N8BR.drop_duplicates(subset='Neighbourhood', keep="first")
df_N8BR=df_N8BR.reset_index(drop=True) #add back the index which is 0,1,2,3,4,5,.... to the first column
df_N8BR.tail()

df_N8BR.sort_values('Neighbourhood', ascending=True)
df_N8BR.reset_index(drop=True).tail() #add back the index which is 0,1,2,3,4,5,.... to the first column

df_N8BR.dtypes

df_N8BR.shape

**----------------------------------------------------------------------------------------------------------------------------------**

**Another way to extract neighbourhood list from website**

**!!! CODE IN THIS SECTION CONTAINS BUGS... THERE IS UNEXPECTED DATA LOSS !!!**

URL = "https://www.epl.ca/blogs/post/edmonton-ward-profiles/"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

#print(soup.prettify()) 

Method 1 for finding neighbors

n8b1=[]
n8b2=[]
for strong_tag in soup.find_all('strong'):
    n8b1.append(strong_tag.text)
    n8b2.append(strong_tag.next_sibling)
    #print(strong_tag.text, strong_tag.next_sibling) #out put is Neighbourhoods:  Aldergrove, Belmead, Britannia, ...

 
column_names = ['#', 'Neighbourhood'] # define the dataframe columns


neighbors = pd.DataFrame(columns=column_names) # instantiate the dataframe (add column name)

for data in range(0, len(n8b2)-1):
    neib = n8b1[data]
    neighbourhood = n8b2[data]

    neighbors = neighbors.append({ '#': neib,
                                   'Neighbourhood': neighbourhood,
                                   }, ignore_index=True)
neighbors.tail(15)

neighbors=neighbors.astype(str)

df=neighbors.copy()
dfn=df[df['#']=='Neighbourhoods:']
dfn

Method 2 for finding neighbors

for length in soup.find_all("strong", text="Neighbourhoods:"):
    print(length.next_sibling.strip())

**---------------------------------------------------------------------------------------------------------------------------------------**

#### **Exam whether the previous data is complete data** compared with another source

This data presents only common neighbour on the official website  
https://www.edmonton.ca/residential_neighbourhoods/your-neighbourhood.aspx    
(or may use https://data.edmonton.ca/Geospatial-Boundaries/Mature-Neighbourhoods-Map/3jmw-i9z8 alternatively)

   
URL = "https://www.edmonton.ca/residential_neighbourhoods/your-neighbourhood.aspx"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

#print(soup.prettify()) 


all_options = soup.find_all('option')

n8b1=[]
for option in all_options:
    if option.text: # skip empty options
        #print('    text:', option.text)                                          #if need, just un-#
        
        #print('   value:', option['value']) # without defaul value                #no-use for this case
        #print('   value:', option.get('value')) # default value `None`            #no-use for this case
        #print('   value:', option.get('value', 'FooBar')) # default value 'FooBar'#no-use for this case
        #print('selected:', option.get('selected')) 
        
        n8b1.append(option.text)
        
        

column_names = ['Neighbourhood'] # define the dataframe columns


neighbors = pd.DataFrame(columns=column_names) # instantiate the dataframe (add column name)

for data in range(0, len(n8b1)-1):
    neib = n8b1[data]

    neighbors = neighbors.append({ 'Neighbourhood': neib,
                                   }, ignore_index=True)

neighbors=neighbors[neighbors['Neighbourhood']!='Choose one...']
neighbors.head()

neighbors.dtypes

#### **Compare 2 dataframe obtained from different sources**

df1=neighbors.copy()
#df1=df1[df1['Neighbourhood']!='Choose one...']
#df1.sort_values('Neighbourhood', ascending=False)
#df1['Neighbourhood'] = df1['Neighbourhood'].apply(lambda x: ' '.join(sorted(x.split())))
df1=df1['Neighbourhood']
df1.head()

df2=df_N8BR.copy()
df2.drop(["Ward"],axis=1, inplace=True)
#df2.sort_values('Neighbourhood', ascending=True)
#df2['Neighbourhood'] = df2['Neighbourhood'].apply(lambda x: ' '.join(sorted(x.split())))
df2=df2['Neighbourhood']
df2.head()

df1[~df1.apply(tuple,1).isin(df2.apply(tuple,1))]

#### **!!!!!!Store dataframe(s) df_N8BR which can be used for other notebooks**

df_N8BR.tail()

%store df_N8BR

**-------------------------------------------------------------------------------------------------------------------------------------**

### 5.1 NEW UPDATE - THE ACCURATE CORRESPONDING 

#### 5.1.1 All Neighbourhoods in Edmonton

This data is obtained from https://data.edmonton.ca/Geospatial-Boundaries/Neighbourhoods-and-Wards/gihh-utrc/data

In [ ]:
df_neighbourhood = pd.read_csv("https://data.edmonton.ca/api/views/gihh-utrc/rows.csv?accessType=DOWNLOAD")
df_neighbourhood.head()

In [ ]:
df_neighbourhood.drop(['Neighbourhood Number','Percentage of Interaction'],axis=1, inplace=True)
df_neighbourhood.rename(columns={'Neighbourhood Name':'Neighbourhood'}, inplace=True)
df_neighbourhood.tail()

In [ ]:
len(df_neighbourhood)

### !!!!!!Store df_neighbourhood (all Neighbourhood V.S. Wards) for other notebook

In [ ]:
df_neighbourhood.sort_values(['Neighbourhood'], inplace=True ,ascending=True)
df_neighbourhood.reset_index(drop=True, inplace=True)
df_neighbourhood.head()

In [ ]:
%store df_neighbourhood

#### 5.1.2 Only Mature Neighbourhoods in Edmonton    
Mature neighbourhoods in Edmonton are those generally completed before 1970, the majority of which experienced their greatest growth in the post-WWII era.

Data is obtained from https://data.edmonton.ca/Geospatial-Boundaries/Mature-Neighbourhoods-Map/3jmw-i9z8

In [ ]:
df_neiborM = pd.read_csv("https://data.edmonton.ca/api/views/gnzw-67i5/rows.csv?accessType=DOWNLOAD")
df_neiborM.head()

In [ ]:
df_neiborM.rename(columns={'Neighbourhood Name':'Neighbourhood'}, inplace=True)

In [ ]:
df_neiborMature=df_neiborM.copy()
df_neiborMature.drop(['the_geom','area_km2','Neighbourhood Number'],axis=1, inplace=True)
df_neiborMature.tail()

In [ ]:

df_neighbourhoodM = pd.merge(df_neighbourhood,
                 df_neiborMature[['Neighbourhood']],
                 on='Neighbourhood')
ID=list(range(1,len(df_neiborMature)+1))
df_neighbourhoodM['NeiborID']=ID
df_neighbourhoodM.tail()

### !!!!!!Store df_neighbourhoodM (Mature Neighbourhood V.S. Wards) for other notebooks

In [ ]:
%store df_neighbourhoodM

### 5.2 Visulization of Neighbourhood

#### 5.2.1 HEAT MAP - ALL Neighbourhoods in Edmonton

Data is obtained from https://data.edmonton.ca/dataset/City-Of-Edmonton-Neighbourhood-Boundaries-Spatial-/tvcx-3vrx

In [ ]:
df_nei=df_neighbourhood.copy()
ID=list(range(1,402))
df_nei['NeiborID']=ID
df_nei.tail()

In [ ]:
# download edmonton geojson file
#!pip install wget
import wget
wget.download('https://data.edmonton.ca/api/geospatial/tvcx-3vrx?method=export&format=GeoJSON', 'edmontonNei.json')
    
print('GeoJSON file downloaded!')

In [ ]:
edmontonNei_geo = r'edmontonNei.json' ######## read geojson file

In [ ]:
# create a plain world map
edmN_map = folium.Map(location=[latitude, longitude], zoom_start=10) #, tiles = 'Stamen Toner')

In [ ]:

# generate choropleth map 
edmN_map.choropleth(
    geo_data=edmontonNei_geo,
    data=df_nei,
    columns=['Neighbourhood', 'NeiborID'],
    key_on='feature.properties.name',
    #threshold_scale=threshold_scale,
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Neighbourhoods in Edmonton'
)

# display map
edmN_map

In [ ]:
import geopandas as gpd
response = requests.get(r"https://data.edmonton.ca/api/geospatial/tvcx-3vrx?method=export&format=GeoJSON")
data = response.json()
NeisA = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

NeisA.tail()

In [ ]:
NeiAmerge=NeisA.copy()
#NeiAmerge=NeiAmerge.sort_values('number', ascending=True)
NeiAmerge=NeiAmerge.sample(frac=1).reset_index(drop=True)

ID=list(range(1,len(NeisA)+1))
NeiAmerge['NeiborID']=ID
NeiAmerge['centroid_long']=NeiAmerge.centroid.x
NeiAmerge['centroid_lag']=NeiAmerge.centroid.y
NeiAmerge.tail()

In [ ]:
import branca.colormap as cm
#colormap = cm.linear.YlGnBu_09 #Paired_12 #Set1_09
colormap = cm.linear.Paired_12.scale(NeiAmerge['NeiborID'].min(), NeiAmerge['NeiborID'].max()).to_step(NeiAmerge['NeiborID'].max()) 
colormap

In [ ]:
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup


# create a plain world map
edmA_map = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='CartoDB positron') #, tiles = 'Stamen Toner')


style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']['NeiborID']), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}


g = folium.GeoJson(
    NeiAmerge,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['name','NeiborID'],
        aliases=['Neighbourhood','Neighbourhood number'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
        sticky=True
        )).add_to(edmA_map)

colormap.add_to(edmA_map)

edmA_map

In [ ]:
class DivIcon(MacroElement):
    def __init__(self, html='', size=(5,5), anchor=(0,0), style=''):
        """TODO : docstring here"""
        super(DivIcon, self).__init__()
        self._name = 'DivIcon'
        self.size = size
        self.anchor = anchor
        self.html = html
        self.style = style

        self._template = Template(u"""
            {% macro header(this, kwargs) %}
              <style>
                .{{this.get_name()}} {
                    {{this.style}}
                    }
              </style>
            {% endmacro %}
            {% macro script(this, kwargs) %}
                var {{this.get_name()}} = L.divIcon({
                    className: '{{this.get_name()}}',
                    iconSize: [{{ this.size[0] }},{{ this.size[1] }}],
                    iconAnchor: [{{ this.anchor[0] }},{{ this.anchor[1] }}],
                    html : "{{this.html}}",
                    });
                {{this._parent.get_name()}}.setIcon({{this.get_name()}});
            {% endmacro %}
            """)
for i in range(len(NeiAmerge)):
    lag=NeiAmerge.iloc[i][6]
    log=NeiAmerge.iloc[i][5]
    text=NeiAmerge.iloc[i][2]
    folium.map.Marker(
        [lag, log],
        icon=DivIcon(
            size=(5,5),
            anchor=(0,0),
            html=text,
            style="""
            font-size:4px;
            background-color: transparent;
            border-color: transparent;
            text-align: center;
            """
        )
    ).add_to(edmA_map)
edmA_map

#### 5.2.2 HEAT MAP - ONLY Mature Neighbourhood in Edmonton

In [ ]:

import geopandas as gpd
response = requests.get(r"https://data.edmonton.ca/api/geospatial/gnzw-67i5?method=export&format=GeoJSON")
data = response.json()
Neis = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

Neis.tail()

In [ ]:
Neimerge=Neis.copy()
ID=list(range(1,len(Neis)+1))
Neimerge['NeiborID']=ID
Neimerge.tail()

In [ ]:
import branca.colormap as cm
#colormap = cm.linear.YlGnBu_09
colormap = cm.linear.YlGnBu_09.scale(Neimerge['NeiborID'].min(), Neimerge['NeiborID'].max()).to_step(len(df_neiborMature)) 
colormap

In [ ]:
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

# create a plain world map
edmM_map = folium.Map(location=[latitude, longitude], zoom_start=10) #, tiles = 'Stamen Toner')


style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']['NeiborID']), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}


g = folium.GeoJson(
    Neimerge,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['name','NeiborID'],
        aliases=['Neighbourhood','Neighbourhood number'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
        sticky=True
        )).add_to(edmM_map)

colormap.add_to(edmM_map)

edmM_map

## Step 6. Neighbourhood classification  
Reference:    
 - https://www.edmonton.ca/city_government/urban_planning_and_design/growth-analysis.aspx?utm_source=virtualaddress&utm_campaign=growthanalysis
 - https://www.edmonton.ca/city_government/urban_planning_and_design/building-permit-activities-visualization.aspx

### 6.1 Import data

In [ ]:
df_class = pd.read_excel("https://www.edmonton.ca/city_government/documents/PDF/Neighbourhood_Building_Permits_2009-2016.xls",'r2016')
df_class.head()

### 6.2 Data cleaning

#### 6.2.a Give each Neighbourhood a Classification ID (2016)

In [ ]:
col_list = ['Neighbourhood Name', 'Typology']
df_class = df_class[col_list]
df_class.rename(columns={'Neighbourhood Name':'Neighbourhood', 'Typology':'Classification'}, inplace=True)
df_class['Neighbourhood']=df_class['Neighbourhood'].str.title()
df_class=df_class[df_class['Neighbourhood']!= 'Grand Total']
df_class.tail()

In [ ]:
df=df_class.copy()
df=df.groupby('Classification',axis=0).count().reset_index()
df.drop(['Neighbourhood'],axis=1, inplace=True)
df

In [ ]:
dclassID=df.copy()
dclassID['ClassificationID']=list(range(1,(len(df)+1)))
dclassID

In [ ]:
dclass=df_class.copy()
mymap =  {'Developing':1, 'Established':2, 'Industrial':3, 'Mature Area':4, 'Mature Area - Central Core':5,'Mature Area - Central Core (Downtown)':6,'Planned':7,'River Valley':8,'TUC':9}
dclass=dclass.applymap(lambda s: mymap.get(s) if s in mymap else s)
dclass.tail()

### !!!!!!Store dclassID Class and its ID

In [ ]:
%store dclassID

##### 6.2.a2 Part New (data from 2019)
Reference:
 - https://www.edmonton.ca/city_government/urban_planning_and_design/building-permit-activities-visualization.aspx
 click the above link, then click net unit growth neighbourhood, then go the the underneath download, click data, right click 'Download all rows as a text file', copy link address.
 Note: The below link may expire for some secure reason.

df_class = pd.read_csv('https://public.tableau.com/vizql/w/BuildingPermitActivitiesMonthly/v/NetUnitGrowthNeighbourhood/vudcsv/sessions/375DFAA4CCFD4C5BA9F43FD85B4EB815-0:0/views/5918020949447600458_11936424844950092242?summary=true')
df_class.tail()

###### M1c.1 Connect to the database from SQL
Let us first load the SQL extension and establish a connection with the database

In [ ]:
%load_ext sql

In [ ]:
%sql ibm_db_sa://gln55437:v6jk%5Es58hvsjd4lj@dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net:50000/BLUDB

Import data from http://ace.edmonton.ca/projects/visualizations/crime-vs-average-assessed-value/   
1. Download Tableau from the website
2. Access 'boundaries+ (crime and assessed tableau)' from tableau and explore all into csv
3. upload to IBM_Cloud→Db2-vs→Manage→Open console→Load data (under GLN55437)

In [ ]:
%sql select * from NEIGHBOURHOOD_MAP LIMIT 5

Import data from SQL SERVER

Identify the database connection credentials

In [ ]:
import ibm_db

In [ ]:
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"
dsn_hostname ="dashdb-txn-sbox-yp-dal09-10.services.dal.bluemix.net"
dsn_port = "50000" 
dsn_protocol = "TCPIP"
dsn_uid ="gln55437"
dsn_pwd ="v6jk^s58hvsjd4lj"     

Create the database connection

In [ ]:
#Create database connection
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

In [ ]:
import pandas
import ibm_db_dbi

In [ ]:
#connection for pandas
pconn = ibm_db_dbi.Connection(conn)

In [ ]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from NEIGHBOURHOOD_MAP"

#retrieve the query results into a pandas dataframe
df_class = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
df_class.head()

In [ ]:
col_list = ['Neighbourhood', 'Neighbourhood_Type']
df_class = df_class[col_list]
df_class.rename(columns={'Neighbourhood_Type':'Classification'}, inplace=True)
#df_class=df_class[df_class['Neighbourhood']!= ""] 
df_class = df_class.dropna()
df_class.tail()

In [ ]:
df=df_class.copy()
df=df.groupby('Classification',axis=0).count().reset_index()

df

In [ ]:
dclass=df_class.copy()
mymap =  {'Developing':1, 'Established':2, 'Industrial':3, 'Mature Area':4, 'Mature Area - Central Core':5,'Mature Area - Central Core (Downtown)':6,'Planned':7,'River Valley':8,'TUC':9}
dclass=dclass.applymap(lambda s: mymap.get(s) if s in mymap else s)
dclass.tail()

#### 6.2.b Mannually add classificationID for missing Neighbourhoods

##### 6.2.b0 Check whether there is unexpected missing data

In [ ]:
dm1=dclass.copy()
dm2=df_neighbourhood.copy()
dm1.drop(['Classification'],axis=1, inplace=True)
dm2.drop(['Ward'],axis=1, inplace=True)
dm2.reset_index(drop=True, inplace=True)

In [ ]:
dmiss=dm2[~dm2.apply(tuple,1).isin(dm1.apply(tuple,1))]

dmiss.reset_index(drop=True, inplace=True)
dmiss.tail()

##### 6.2.b1 Check whether there is unexpected missing data (new)

In [ ]:
import geopandas as gpd
response = requests.get(r"https://data.edmonton.ca/api/geospatial/tvcx-3vrx?method=export&format=GeoJSON")
data = response.json()
neighbourhood = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

neighbourhood.tail()

In [ ]:
neighbourhood.rename(columns={'name':'Neighbourhood'}, inplace=True)

In [ ]:
dm1=dclass.copy()
dm2=neighbourhood.copy()
dm1.drop(['Classification'],axis=1, inplace=True)
dm2.drop(['geometry','area_km2','number'],axis=1, inplace=True)
dm2.reset_index(drop=True, inplace=True)

In [ ]:
dmiss=dm2[~dm2.apply(tuple,1).isin(dm1.apply(tuple,1))]
dmiss.sort_values(['Neighbourhood'], inplace=True ,ascending=True)
dmiss.reset_index(drop=True, inplace=True)
dmiss.tail()

##### 6.2.b2 Part A keyword check

In [ ]:
a=dmiss['Neighbourhood'].str.strip().str[-10:-1].iloc[43]
print(a)

In [ ]:
a=dmiss['Neighbourhood'].iloc[0]
print(a)

In [ ]:
a=dmiss['Neighbourhood'].str.strip().str[0:8].iloc[1]
print(a)

In [ ]:
a=dmiss['Neighbourhood'].str.strip().str[0:12].iloc[44]
print(a)

In [ ]:
a=dmiss['Neighbourhood'].str.contains('River Valley', regex=False).iloc[44]
print(a)

In [ ]:
dmiss1a=[]
dmiss1b=[]
for i in range((len(dmiss)-1)):
    #a=dmiss['Neighbourhood'].str.strip().str[-10:-1].iloc[i]
    a=dmiss['Neighbourhood'].str.contains('Industrial', regex=False).iloc[i]
    b=dmiss['Neighbourhood'].iloc[i]
    c=dmiss['Neighbourhood'].str.strip().str[0:7].iloc[i]
    d=dmiss['Neighbourhood'].str.strip().str[0:12].iloc[i]
    if a == True:  
        dmiss1a.append(b)
        dmiss1b.append(3)
    elif c == 'Anthony':
        dmiss1a.append(b)
        dmiss1b.append(9)
    elif d=='River Valley':
        dmiss1a.append(b)
        dmiss1b.append(8)
    else:
        continue
    

In [ ]:
dataM = {'Neighbourhood': dmiss1a,
            'Classification':dmiss1b,
        }

dtM = pd.DataFrame (dataM, columns = ['Neighbourhood','Classification'])
dtM.tail()

##### 6.2.b3 Part B manually add (based on 2017 data)
References:
 - https://public.tableau.com/profile/city.of.edmonton#!/vizhome/NeighbourhoodProfiles_FederalCensus2016/PopulationbyAgeandGender
 - https://www.edmonton.ca/city_government/documents/Mature_Neighbourhood_Reinvestment_Report_2017.pdf

In [ ]:
dm1=dmiss.copy()
dm2=dtM.copy()
dm2.drop(['Classification'],axis=1, inplace=True)

In [ ]:
dmiss2=dm1[~dm1.apply(tuple,1).isin(dm2.apply(tuple,1))]

dmiss2.reset_index(drop=True, inplace=True)
dmiss2.tail()

In [ ]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from NEIGHBOURHOOD_ADDON"

#retrieve the query results into a pandas dataframe
df_addon = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
df_addon.head()

In [ ]:
df_addon =df_addon[df_addon['Classification']!= "Not assigned"] 
df_addon.reset_index(drop=True, inplace=True)
df_addon.tail()

In [ ]:
df_addon["Classification"]=df_addon["Classification"].astype(int)
#df_addon.astype(int)
df_addon.dtypes

##### 6.2.b4 Part C Merge

In [ ]:
dclass1=dclass.copy()
dclass1 = dclass1.append([dtM,df_addon], ignore_index=True)
#dclass1 =dclass1[dclass1['Classification']!= "University Of Alberta"] 
dclass1.sort_values(['Neighbourhood'], inplace=True ,ascending=True)
dclass1.reset_index(drop=True, inplace=True)
dclass1.tail()

### !!!!!!Store dclass1 Neighbourhood and its classification ID

In [ ]:
%store dclass1

### 6.3 HEAT MAP - Neighbourhood classification

In [ ]:
import geopandas as gpd
response = requests.get(r"https://data.edmonton.ca/api/geospatial/tvcx-3vrx?method=export&format=GeoJSON")
data = response.json()
NeisA = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

NeisA.tail()

In [ ]:
NeisA.rename(columns={'name':'Neighbourhood'}, inplace=True)
NeiAmerge = pd.merge(NeisA,
                dclass1[['Neighbourhood','Classification']],
                 on='Neighbourhood')

NeiAmerge.tail()

In [ ]:
import branca.colormap as cm
#colormap = cm.linear.YlGnBu_09
colormap = cm.linear.Paired_09.scale(NeiAmerge['Classification'].min(),NeiAmerge['Classification'].max()).to_step(NeiAmerge['Classification'].max()) 
colormap

In [ ]:
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup


# create a plain world map
edmA_map = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='CartoDB positron') #, tiles = 'Stamen Toner')


style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']['Classification']), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}


g = folium.GeoJson(
    NeiAmerge,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Neighbourhood','Classification'],
        aliases=['Neighbourhood','Classification number'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
        sticky=True
        )).add_to(edmA_map)

colormap.add_to(edmA_map)

edmA_map

In [ ]:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Neighbourhood Classification (2017)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#a6cee3;opacity:0.7;'></span>Developing</li>
    <li><span style='background:#1f78b4;opacity:0.7;'></span>Established</li>
    <li><span style='background:#b2df8a;opacity:0.7;'></span>Industrial</li>
    <li><span style='background:#33a02c;opacity:0.7;'></span>Mature Area</li>
    <li><span style='background:pink;opacity:0.7;'></span>Mature Area-Central Core</li>
    <li><span style='background:red;opacity:0.7;'></span>Mature Area-Central Core (Downtown)</li>
    <li><span style='background:sandybrown;opacity:0.7;'></span>Planned</li>
    <li><span style='background:darkorange;opacity:0.7;'></span>River Valley</li>
    <li><span style='background:plum;opacity:0.7;'></span>Transportation and Utility Corridor</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

edmA_map.get_root().add_child(macro)

edmA_map

In [ ]:
edmA_map.save('Classification_neighbourhood.html')

### 6.4 Only conside mature neighbourhood, estblished neighbourhood and developing neighbourhood

#### 6.4.1 Data clean

In [ ]:
dclassMED=dclass1.copy()
dclassMED=dclassMED[dclassMED['Classification']!= 3]
dclassMED=dclassMED[dclassMED['Classification']!= 7]
dclassMED=dclassMED[dclassMED['Classification']!= 8]
dclassMED=dclassMED[dclassMED['Classification']!= 9]
dclassMED.reset_index(drop=True, inplace=True)
dclassMED.tail()

### !!!!!!Store dclassMED: Mature, Established, Developing Neighbourhood and its classification ID

In [ ]:
%store dclassMED

#### 6.4.2 HEAT MAP - Neighbourhood classification

In [ ]:
import geopandas as gpd
response = requests.get(r"https://data.edmonton.ca/api/geospatial/tvcx-3vrx?method=export&format=GeoJSON")
data = response.json()
NeisA = gpd.GeoDataFrame.from_features(data, crs='EPSG:4326')

NeisA.tail()

In [ ]:
NeisA.rename(columns={'name':'Neighbourhood'}, inplace=True)
NeiAmerge = pd.merge(NeisA,
                dclassMED[['Neighbourhood','Classification']],
                 on='Neighbourhood')

NeiAmerge.tail()

In [ ]:
import branca.colormap as cm
#colormap = cm.linear.YlGnBu_09
colormap = cm.linear.Paired_06.scale(NeiAmerge['Classification'].min(),NeiAmerge['Classification'].max()).to_step(NeiAmerge['Classification'].max()) 
colormap

In [ ]:
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup


# create a plain world map
edmA_map = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='CartoDB positron') #, tiles = 'Stamen Toner')
colormap.caption = 'Classification ID'
#:Developing:1, Established:2, Industrial:3, \n Mature Area:4,Mature Area - Central Core:5'
#Mature Area - Central Core (Downtown):6, 
#Planned:7, River Valley:8,TUC:9'''

style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']['Classification']), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}


g = folium.GeoJson(
    NeiAmerge,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Neighbourhood','Classification'],
        aliases=['Neighbourhood','Classification number'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
        sticky=True
        )).add_to(edmA_map)

colormap.add_to(edmA_map)

edmA_map

In [ ]:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Neighbourhoods for further analysis</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#a6cee3;opacity:0.7;'></span>Developing</li>
    <li><span style='background:#1f78b4;opacity:0.7;'></span>Established</li>
    <li><span style='background:#33a02c;opacity:0.7;'></span>Mature Area</li>
    <li><span style='background:pink;opacity:0.7;'></span>Mature Area-Central Core</li>
    <li><span style='background:red;opacity:0.7;'></span>Mature Area-Central Core (Downtown)</li>


  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

edmA_map.get_root().add_child(macro)

edmA_map

In [ ]:
edmA_map.save('DEMCneighbourhood.html')

## **End of Main Script**